In [1]:
%load_ext autoreload
%autoreload 2

In this example we will create alkyl chains of various lengths and put them in a simulation cell to be analyzed 

In [2]:
from __future__ import division, unicode_literals

In [3]:
import os 
from pprint import pprint

In [4]:
import streamm

Now let's create project and resource to keep track of our work

In [5]:
alkyl_example = streamm.Project('alkyl_example')
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [6]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [7]:
print res_local.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Create the local directories that will store our files 

In [8]:
res_local.make_dir()

Tell the project about our directories 

In [9]:
alkyl_example.set_resource(res_local)

Read in the methane.xyz file created in the structure.ipynb example 

In [10]:
methane = streamm.Buildingblock('methane')

In [11]:
methane.read_xyz()

In [12]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

In [13]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [14]:
print methane.n_particles

5


In [15]:
print methane.print_properties()

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


In [16]:
for pkey,p in methane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [17]:
for pk,p in methane.particles.iteritems():
    p.residue = 1
    p.resname = 'METH'

In [18]:
methane.particles[1].rsite = 'RH'
methane.particles[2].rsite = 'RH'

In [19]:
methane.find_rsites()

In [20]:
print methane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[2] H (H) index:2 n_bonds:1] 



In [21]:
ethane = streamm.Buildingblock('ethane')

In [22]:
ethane.read_xyz()

In [23]:
ethane.bonded_nblist = ethane.guess_nblist(0,radii_buffer=1.25)

In [24]:
ethane.bonded_bonds()
ethane.bonded_angles()
ethane.bonded_dih()

In [25]:
print ethane.n_particles

8


In [26]:
print ethane.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


In [27]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [28]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

In [29]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

In [30]:
ethane.find_rsites()

In [31]:
print ethane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[5] H (H) index:5 n_bonds:1] 



In [32]:
import copy

In [33]:
hexyl = copy.deepcopy(ethane)

In [34]:
from streamm.structures.buildingblock import attach

In [35]:
for i in range(3):
    hexyl = attach(hexyl,ethane,'RH',1,'RH',0)

In [36]:
hexyl.tag = 'hexyl'

In [37]:
hexyl.write_xyz()

In [38]:
print hexyl.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[23] H (H) index:23 n_bonds:1] 



In [39]:
print hexyl.particles[14].symbol

H


In [40]:
hexyl.particles[14].rsite = 'R2'

In [41]:
hexyl.find_rsites()

In [42]:
ethyl_hexyl = attach(hexyl,ethane,'R2',0,'RH',0)

In [43]:
ethyl_hexyl.tag = 'ethyl_hexyl'

In [44]:
ethyl_hexyl.write_xyz()

In [45]:
for pk,p in ethyl_hexyl.particles.iteritems():
    p.residue = 1
    p.resname = 'ETHHEX'

In [46]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [47]:
print oplsaa


    Parameters 
      LJ parameters 2 
      Bond parameters 3 
      Angle parameters 2 
      Dihedral parameters 1 
      Imporper Dihedral parameters 0 



In [48]:
nwchem_i = streamm.NWChem('nw_ethane_HF')

In [49]:
alkyl_example.add_calc(nwchem_i)

In [50]:
nwchem_i.strucC = ethane

In [51]:
nwchem_i.set_resource(res_local)

In [52]:
nwchem_i.make_dir()

In [53]:
os.chdir(nwchem_i.dir['scratch'])

In [54]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [55]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [56]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

In [57]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_i.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [58]:
pprint(nwchem_i.properties)

{u'allocation': u'',
 u'basis': u'6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/nw_ethane_HF/',
 u'spin_mult': 1,
 u'task': u'SCF ',
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


In [59]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

In [60]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_i.tag))

In [61]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

In [62]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.dump_json()

In [63]:
os.chdir(nwchem_i.dir['scratch'])

In [64]:
nwchem_i.run()

In [65]:
alkyl_example.check()

Calculation nw_ethane_HF has status running


In [66]:
nwchem_i.analysis()

In [67]:
nwchem_i.store()

In [68]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.dump_json()

In [69]:
gaussian_i = streamm.Gaussian('gaus_ethane_HF')

In [70]:
alkyl_example.add_calc(gaussian_i)

In [71]:
gaussian_i.strucC = ethane

In [72]:
gaussian_i.set_resource(res_local)

In [73]:
gaussian_i.make_dir()

In [74]:
os.chdir(gaussian_i.dir['scratch'])

In [75]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [76]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [77]:
gaussian_i.load_str('templates','com')        
gaussian_i.load_str('templates','run')

In [78]:
gaussian_i.properties['commands'] = 'HF/3-21G SP'
gaussian_i.properties['method'] = 'UHF'
gaussian_i.properties['charge'] = 0
gaussian_i.properties['spin_mult'] = 1
gaussian_i.properties['coord'] = gaussian_i.strucC.write_coord()

In [79]:
pprint(gaussian_i.properties)

{u'allocation': u'',
 u'charge': 0,
 u'commands': u'HF/3-21G SP',
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Normal termination of Gaussian',
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/gaus_ethane_HF/',
 u'spin_mult': 1,
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


In [80]:
gaussian_i.replacewrite_prop('com','input','com','%s.com'%(gaussian_i.tag))

In [81]:
gaussian_i.properties['input_com'] = gaussian_i.files['input']['com']
gaussian_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaussian_i.tag))

In [82]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaussian_i.tag)
gaussian_i.add_file(file_type,file_key,file_name)

In [83]:
os.chdir(gaussian_i.dir['home'])
alkyl_example.dump_json()

In [84]:
os.chdir(gaussian_i.dir['scratch'])

In [85]:
gaussian_i.run()

In [86]:
alkyl_example.check()

Calculation nw_ethane_HF has status running
Calculation gaus_ethane_HF has status running


In [87]:
os.chdir(alkyl_example.dir['home'])
alkyl_example.dump_json()

In [88]:
lmp_ethane = streamm.LAMMPS('lmp_ethane')

In [89]:
alkyl_example.add_calc(lmp_ethane)

In [90]:
lmp_ethane.set_resource(res_local)

In [91]:
lmp_ethane.make_dir()

In [92]:
lmp_ethane.paramC = oplsaa

In [93]:
import streamm

In [94]:
lmp_ethane.strucC =  streamm.Buildingblock(matrix=[50.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,50.0])

In [95]:
seed = 92734
lmp_ethane.strucC = streamm.add_struc(lmp_ethane.strucC,ethyl_hexyl,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


In [96]:
lmp_ethane.strucC.tag ='ethyl_hexyl_x10'

In [97]:
lmp_ethane.strucC.write_xyz()

In [98]:
seed = 283674
lmp_ethane.strucC = streamm.add_struc(lmp_ethane.strucC,ethane,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


In [99]:
lmp_ethane.strucC.tag += '_ethane_x10'

In [100]:
lmp_ethane.strucC = streamm.add_struc_grid(lmp_ethane.strucC,methane,50)

In [101]:
lmp_ethane.strucC.tag += '_methane_x50'

In [102]:
lmp_ethane.strucC.write_xyz()

In [103]:
lmp_ethane.set_ffparam()

In [104]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_sp.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_ethane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [105]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_ethane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [106]:
os.chdir(lmp_ethane.dir['scratch'])

In [107]:
lmp_ethane.load_str('templates','in')
lmp_ethane.load_str('templates','run')

In [108]:
lmp_ethane.write_data()

In [109]:
lmp_ethane.replacewrite_prop('in','input','in','%s.in'%(lmp_ethane.tag))

In [110]:
lmp_ethane.properties['input_in'] = lmp_ethane.files['input']['in']
lmp_ethane.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_ethane.tag))

In [111]:
os.chdir(lmp_ethane.dir['home'])
lmp_ethane.dump_json()

In [112]:
lmp_ethane.run()

In [113]:
os.chdir(lmp_ethane.dir['scratch'])
lmp_ethane.check()

In [114]:
pprint("Calculation:{} has status:{}".format(lmp_ethane.tag,lmp_ethane.meta['status']))

u'Calculation:lmp_ethane has status:written'


In [124]:
groups_i = streamm.Groups('residues',lmp_ethane.strucC)
groups_i.group_prop('residue','res')

In [126]:
groups_i.write_cm_xyz()

In [127]:
groups_i.find_pairs()

TypeError: find_pairs() takes at least 3 arguments (1 given)

In [125]:
for g_index,group_i in groups_i.groups.iteritems():
    print group_i.pkeys

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 320, 321, 322, 323, 324, 325, 326, 327, 400, 401, 402, 403, 404]
[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 328, 329, 330, 331, 332, 333, 334, 335, 405, 406, 407, 408, 409]
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 336, 337, 338, 339, 340, 341, 342, 343, 410, 411, 412, 413, 414]
[96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 344, 345, 346, 347, 348, 349, 350, 351, 415, 416, 417, 418, 419]
[128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 352, 353, 354, 355, 356, 357, 358, 359, 420, 

Awesome! 